# Pericyte Analysis

This notebook quantifies the fraction of blood vessels covered by pericytes. The code reads the blood vessels and pericytes segmented using the code "process_images.py" and measures the overlap between them using a distance transform.

In [ ]:
import numpy as np
import scipy.ndimage as ndi
from igraph import *
import os
import readimg
import natsort
import pickle
import cv2
                                                                 
def graph_to_img(g,shape):

	node_pos = g.vs['pos_no']
	ed_pos = g.es['pos_ed']
	img = np.zeros(shape, dtype=int)
	for i in range(len(node_pos)):
		node = node_pos[i]
		if (np.isscalar(node[0])==True):
			img[node[0],node[1],node[2]] = 1
		else:
			for pos in node:
				img[pos[0],pos[1],pos[2]] = 1
			
	for i in range(len(ed_pos)):		
		edge = ed_pos[i]
		for point in edge:
			img[point[0],point[1],point[2]] = 1

	return img

def get_files(inFolderOriginal, batchName):
  
    dataTree = os.walk(inFolderOriginal)
    files = []
    for Pfolder, subFolders, subFolderFiles in dataTree:
        if len(subFolderFiles)>0:
            ind = Pfolder.find(batchName)+len(batchName)+1
            if os.name=='nt':
                folders = Pfolder[ind:].split('\\') 
            else:
                folders = Pfolder[ind:].split('/') 

            if ((folders[-1]!='ROIs') and (folders[-1]!='maximum_projection') and (folders[-1]!='equalized')):
                if (len(folders[0])>0):
                    folders = [folder+'/' for folder in folders]

                for subFolderFile in subFolderFiles:
                    if '.tif' not in subFolderFile:
                        files.append(folders+[subFolderFile])
    files = natsort.natsorted(files)	
    
    return files
    
def measure_length(g, scale, img_ROI):

    tam = 0
    for i in range(g.ecount()):
        ed = np.array(g.es['pos_ed'][i])

        is_inside_ROI = img_ROI[ed[:,0],ed[:,1],ed[:,2]]
        is_inside_ROI = np.tile(is_inside_ROI[1:],(3,1)).T

        ded = np.diff(ed,axis=0)
        ded *= is_inside_ROI

        tam_aux = np.zeros(ded.shape[0])
        for j in range(ded.shape[0]):
            tam_aux[j] = np.sqrt(np.sum((ded[j]*scale)**2))

        tam += np.sum(tam_aux)

    tam *= 1e-3
        
    return tam

## Calculate values

In [ ]:
# Calculate values
batchNameVessel = 'Vascular Project_Picture CD31-PDGFR-vGLUT2-IBA1'
batchNamePericytes = 'Vascular Project_Picture CD31-PDGFR-vGLUT2-IBA1_pericytes'

inFolderVessNetwork = '../data/Network/'+batchNameVessel+'/'
inFolderPeriNetwork = '../data/Network/'+batchNamePericytes+'/'
inFolderOriginal = '/media/Data/cesar/Baptiste data/Project neuroinflammation/'+batchNameVessel+'/'
inFolderVessBinary = '../data/Binary/'+batchNameVessel+'/'
inFolderPeriBinary = '../data/Binary/'+batchNamePericytes+'/'
#outFolder = '../data/'

distThresh = 4

files = get_files(inFolderOriginal, batchNameVessel)

list_fracVolVol = []
list_fracGraphVolLen = []
list_fracGraphGraphLen = []
for q in range(len(files)):
    
    subFolder = ''.join(files[q][0:-1])
    originalFolder = inFolderOriginal+subFolder
    ROIFolder = inFolderOriginal+'ROIs/'
    binaryFolderVess = inFolderVessBinary+subFolder
    binaryFolderPeri = inFolderPeriBinary+subFolder
    networkFolderVess = inFolderVessNetwork+subFolder+'core/numpy/'
    networkFolderPeri = inFolderPeriNetwork+subFolder+'core/numpy/'


    ind = files[q][-1].rfind('.')
    fileName = files[q][-1][0:ind]
    print("\nAnalyzing image \""+str(fileName)+"\" ("+str(q+1)+" of "+str(len(files))+")")

    img_np_or,scale_or = readimg.ReadImg(originalFolder+files[q][-1], 1)
    scale = [np.min(scale_or)]*3	

    file = binaryFolderVess+'numpy/'+fileName+'.npy'
    imgBinVess = np.load(file)
    file = binaryFolderPeri+'numpy/'+fileName+'.npy'
    imgBinPeri = np.load(file)
    
    size_z,size_x,size_y = imgBinVess.shape

    file = networkFolderVess+fileName+'.npy'
    gVess = np.load(file, encoding='latin1').item()
    file = networkFolderPeri+fileName+'.npy'
    gPeri = np.load(file, encoding='latin1').item()

    # Read ROI
    arq_name = ROIFolder+fileName+'.png'
    if (os.path.isfile(arq_name)):
        img_ROI = (cv2.imread(arq_name,0).astype(np.int)>0)*1
    else:
        img_ROI = np.ones([size_x,size_y],dtype=int)
        
    img_ROI = np.tile(img_ROI, (imgBinVess.shape[0], 1, 1))    
        
    imgGraphVess = graph_to_img(gVess, imgBinVess.shape)
    imgGraphPeri = graph_to_img(gPeri, imgBinPeri.shape)
    
    imgBinVess = imgBinVess*img_ROI
    imgBinPeri = imgBinPeri*img_ROI
    imgGraphVess = imgGraphVess*img_ROI
    imgGraphPeri = imgGraphPeri*img_ROI
    
    imgVessAndPeri = np.logical_and(imgBinVess, imgBinPeri)
    fracVolVol = np.sum(imgVessAndPeri)/np.sum(imgBinVess)
    
    imgGraphVessAndPeri = np.logical_and(imgGraphVess, imgBinPeri)

    # Calculate overlap
    imgDistPeri = ndi.distance_transform_edt(1-imgGraphPeri)
    imgDistAnd = imgDistPeri*imgGraphVess
    imgDistAndSmall = (imgDistAnd<distThresh)*imgGraphVess
    
    lengthVess = measure_length(gVess, scale, img_ROI)
    
    lengthGraphVessAndPeri = measure_length(gVess, scale, imgGraphVessAndPeri)
    fracGraphVolLen = lengthGraphVessAndPeri/lengthVess
    
    lengthGraphVessAndGraphPeri = measure_length(gVess, scale, imgDistAndSmall)
    fracGraphGraphLen = lengthGraphVessAndGraphPeri/lengthVess
    
    list_fracVolVol.append(fracVolVol)                # Volume of overlap (number of pixels) divided by blood vessel volume
    list_fracGraphVolLen.append(fracGraphVolLen)      # Length of blood vessels that overlap with segmented pericytes divided by vessel length
    list_fracGraphGraphLen.append(fracGraphGraphLen)  # Length of blood vessels that are at most 4 pixels from the closest pericyte medial axis divided by vessel length

    pickle.dump([list_fracVolVol, list_fracGraphVolLen, list_fracGraphGraphLen], open('pericyte_quantifications.dat', 'wb'))

## Analyse

In [ ]:
list_fracVolVol, list_fracGraphVolLen, list_fracGraphGraphLen = pickle.load(open('pericyte_quantifications.dat', 'rb'))
outFolder = '../results/'

results = 'Name,Fraction of vessel length covered by pericytes, Fraction of vessel volume covered by pericytes\n'
for idx, (file, fracVolVol, fracGraphVolLen) in enumerate(zip(files, list_fracVolVol, list_fracGraphVolLen)):

    ind = files[idx][-1].rfind('.')
    fileName = files[idx][-1][0:ind]
    
    results += f'{fileName},{fracGraphVolLen:.2f},{fracVolVol:.2f}\n'
    
open(f'{outFolder}{batchNamePericytes}_results.txt', 'w').write(results)